# Based on [this](https://machinelearningmastery.com/multi-class-classification-tutorial-keras-deep-learning-library/) tutorial

In [2]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

Using TensorFlow backend.


In [ ]:
train_data_dir = 'dankKnowledge/trainingImages/'
validation_data_dir = 'dankKnowledge/testingImages/'
nb_train_samples = 400
nb_validation_samples = 100
epochs = 10
batch_size = 16

if K.image_data_format() == 'channels_first': 
	input_shape = (3, None, None) 
else: 
	input_shape = (None, None, 3) 

model = Sequential() 
model.add(Conv2D(32, (2, 2), input_shape = input_shape))
model.add(Activation('relu'))
model.add(AveragePooling2D((2, 2)))

model.add(Conv2D(64, (2, 2)))
model.add(Activation('relu'))
model.add(GlobalAveragePooling2D(K.image_data_format()))

model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.summary()

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

train_datagen = ImageDataGenerator(
				rescale = 1. / 255,
				shear_range = 0.2,
				zoom_range = 0.2,
			horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1. / 255)

train_generator = train_datagen.flow_from_directory(train_data_dir,
    batch_size = batch_size, class_mode ='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    batch_size = batch_size, class_mode ='binary')

model.fit_generator(train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs, validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size)

model.save('MultiMemeModel.h5')